In [1]:
import os,sys
import warnings
sys.path.insert(1, os.path.join(os.getcwd()  , '..'))
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

In [2]:
import shallowsim as sb
import pandas as pd
import math

In [3]:
args = sb.ModelArgs()
c = sb.Config()
c.bs_list=[16,32,64,128,256,512,1024,2048]
gpu_blackwell_decode = sb.get_gpu_info('./device/gpu_info.csv', 
                                    decoding_mode=True, print_console=True) 

#gpu_blackwell_decode['H20'].pcie_bw = 25 #change bw to 1.6Tbps..

| gpu_type      |   sm |   comm_sm |   fp16 |   fp8 |   fp4 |   mem |   mem_bw |   nvlink_bw |   pcie_bw |   gpu_per_node |
|:--------------|-----:|----------:|-------:|------:|------:|------:|---------:|------------:|----------:|---------------:|
| DGX-B300      |  160 |        20 |   3375 |  7500 | 15000 |   288 |     8000 |         900 |       100 |              8 |
| DGX-B200      |  160 |        20 |   2250 |  4500 |  9000 |   180 |     8000 |         900 |       100 |              8 |
| GB200-NVL72   |  160 |        20 |   2500 |  5000 | 10000 |   192 |     8000 |         900 |       100 |             72 |
| GB300-NVL72   |  160 |        20 |   3750 |  7500 | 15000 |   288 |     8000 |         900 |       100 |             72 |
| Rubin-NVL144  |  110 |        12 |   6400 | 12800 | 25600 |   144 |     6500 |         900 |       100 |            144 |
| RubinU-NVL576 |  110 |        12 |   6500 | 13000 | 26000 |   256 |     8000 |        1350 |       100 |            576 |
| H200  

In [6]:
dfs = sb.decode_time_with_ep_list(args,gpu_blackwell_decode,c,print_console=False,fp8_combine=True,tps_limit=0)

# 推理性能天梯榜

In [7]:
dfs_o = dfs.groupby(['GPU','BatchSize'],as_index=False).apply(lambda t: t[t.Total==t.Total.max()]).sort_values(['Total'],ascending=False).reset_index(drop=True)
dfs_o.style.bar(subset=['TPS','Total'],color='#6495ED')\
      .applymap(sb.gpu_category_color,props=sb.gpu_category_idx(gpu_blackwell_decode),subset=['GPU'])\
      .applymap(sb.color_positive_red, subset=['Delta'])\
      .background_gradient(subset=['Comm_Impact'],cmap=sb.cm)\
      .format(precision=3) 


,GPU,TP,EP,BatchSize,DenseMLA,DenseMLP,SparseMLA,Combine,SharedExpert,RoutedExpert,Dispatch,COMP_SUM,COMM_SUM,Delta,TPOT,TPOT_O,TPS,TPS_O,Total,Total_O,Comm_Impact
0,RubinU-NVL576,4,320,256,0.187,0.065,0.181,0.022,0.008,0.018,0.022,0.207,0.044,-0.144,12.747,12.747,78.451,78.451,20083.367,20083.367,0.000
1,RubinU-NVL576,1,320,1024,0.749,0.092,0.799,0.059,0.012,0.053,0.059,0.864,0.118,-0.694,52.639,52.639,18.997,18.997,19453.097,19453.097,0.000
2,RubinU-NVL576,1,320,512,0.375,0.074,0.425,0.034,0.009,0.029,0.034,0.463,0.069,-0.365,28.211,28.211,35.447,35.447,18148.734,18148.734,0.000
3,Rubin-NVL144,1,144,512,0.419,0.087,0.469,0.047,0.011,0.039,0.047,0.518,0.093,-0.386,31.561,31.561,31.685,31.685,16222.501,16222.501,0.000
4,GB300-NVL72,4,72,256,0.260,0.072,0.201,0.028,0.009,0.048,0.028,0.258,0.057,-0.153,15.976,15.976,62.595,62.595,16024.258,16024.258,0.000
5,RubinU-NVL576,8,320,128,0.094,0.060,0.117,0.016,0.007,0.012,0.016,0.136,0.032,-0.092,8.342,8.342,119.876,119.876,15344.152,15344.152,0.000
6,Rubin-NVL144,1,144,256,0.209,0.078,0.259,0.028,0.009,0.027,0.028,0.295,0.057,-0.212,17.995,17.995,55.572,55.572,14226.543,14226.543,0.000
7,GB300-NVL72,1,72,1024,1.039,0.119,1.089,0.083,0.016,0.106,0.083,1.211,0.166,-0.939,73.703,73.703,13.568,13.568,13893.519,13893.519,0.000
8,GB300-NVL72,1,72,512,0.520,0.087,0.570,0.047,0.011,0.065,0.047,0.646,0.093,-0.488,39.281,39.281,25.458,25.458,13034.396,13034.396,0.000
9,Rubin-NVL144,4,144,128,0.105,0.073,0.134,0.019,0.008,0.022,0.019,0.165,0.038,-0.104,10.096,10.096,99.048,99.048,12678.178,12678.178,0.000


# 查看单个CPU的性能

In [15]:
sb.df_filter(dfs,'Rubin-NVL144',0).style\
      .bar(subset=['TPS','Total'],color='#6495ED')\
      .applymap(sb.color_positive_red, subset=['Delta'])\
      .background_gradient(subset=['Comm_Impact'],cmap=sb.cm)\
      .format(precision=3) 

,GPU,TP,EP,BatchSize,DenseMLA,DenseMLP,SparseMLA,Combine,SharedExpert,RoutedExpert,Dispatch,COMP_SUM,COMM_SUM,Delta,TPOT,TPOT_O,TPS,TPS_O,Total,Total_O,Comm_Impact
46,Rubin-NVL144,1,8,16,0.013,0.069,0.063,0.011,0.009,0.254,0.011,0.326,0.022,-0.050,19.131,19.131,52.272,52.272,836.344,836.344,0.000
47,Rubin-NVL144,4,8,16,0.013,0.069,0.074,0.011,0.009,0.254,0.011,0.336,0.022,-0.060,19.745,19.745,50.647,50.647,810.345,810.345,0.000
48,Rubin-NVL144,8,8,16,0.013,0.069,0.073,0.011,0.009,0.254,0.011,0.335,0.022,-0.059,19.699,19.699,50.763,50.763,812.215,812.215,0.000
49,Rubin-NVL144,1,8,32,0.026,0.070,0.076,0.012,0.010,0.264,0.012,0.350,0.025,-0.062,20.607,20.607,48.528,48.528,1552.898,1552.898,0.000
50,Rubin-NVL144,4,8,32,0.026,0.070,0.082,0.012,0.010,0.264,0.012,0.357,0.025,-0.068,20.964,20.964,47.700,47.700,1526.408,1526.408,0.000
51,Rubin-NVL144,1,8,64,0.052,0.071,0.102,0.015,0.011,0.269,0.015,0.382,0.029,-0.084,22.544,22.544,44.358,44.358,2838.901,2838.901,0.000
52,Rubin-NVL144,1,8,128,0.105,0.073,0.155,0.019,0.013,0.285,0.019,0.452,0.038,-0.129,26.757,26.757,37.374,37.374,4783.864,4783.864,0.000
148,Rubin-NVL144,1,16,16,0.013,0.069,0.063,0.011,0.009,0.132,0.011,0.204,0.022,-0.050,12.076,12.076,82.806,82.806,1324.899,1324.899,0.000
149,Rubin-NVL144,4,16,16,0.013,0.069,0.074,0.011,0.009,0.132,0.011,0.215,0.022,-0.060,12.690,12.690,78.801,78.801,1260.815,1260.815,0.000
150,Rubin-NVL144,8,16,16,0.013,0.069,0.073,0.011,0.009,0.132,0.011,0.214,0.022,-0.059,12.645,12.645,79.084,79.084,1265.349,1265.349,0.000


In [16]:
gpu = 'H20'
tps_limit = 1

tdf = sb.df_filter(dfs,gpu,tps_limit=tps_limit)
sb.df_sort(tdf,value='Total',ascending=False).style\
     .bar(subset=['TPS','Total'],color='#6495ED')\
     .applymap(sb.color_positive_red, subset=['Delta'])\
     .background_gradient(subset=['Comm_Impact'],cmap=sb.cm)\
     .format(precision=3) 

,GPU,TP,EP,BatchSize,DenseMLA,DenseMLP,SparseMLA,Combine,SharedExpert,RoutedExpert,Dispatch,COMP_SUM,COMM_SUM,Delta,TPOT,TPOT_O,TPS,TPS_O,Total,Total_O,Comm_Impact
0,H20,8,320,32,0.630,0.212,0.164,0.092,0.036,0.114,0.051,0.315,0.144,-0.057,20.771,20.771,48.144,48.144,1540.601,1540.601,0.000
1,H20,4,320,64,1.208,0.313,0.403,0.175,0.047,0.204,0.092,0.654,0.267,-0.183,42.475,42.475,23.543,23.543,1506.768,1506.768,0.000
2,H20,4,144,64,1.208,0.313,0.403,0.175,0.047,0.229,0.092,0.678,0.267,-0.183,43.908,43.908,22.775,22.775,1457.594,1457.594,0.000
3,H20,8,144,32,0.630,0.212,0.164,0.092,0.037,0.149,0.051,0.350,0.144,-0.058,22.854,22.854,43.756,43.756,1400.204,1400.204,0.000
4,H20,4,72,64,1.208,0.313,0.403,0.175,0.050,0.298,0.092,0.750,0.267,-0.185,48.073,48.073,20.802,20.802,1331.299,1331.299,0.000
5,H20,8,72,32,0.630,0.212,0.164,0.092,0.039,0.211,0.051,0.414,0.144,-0.059,26.532,26.532,37.691,37.691,1206.096,1206.096,0.000
6,H20,4,36,64,1.208,0.313,0.403,0.175,0.053,0.422,0.092,0.877,0.267,-0.188,55.429,55.429,18.041,18.041,1154.620,1154.620,0.000
7,H20,8,320,16,0.341,0.162,0.118,0.051,0.031,0.074,0.031,0.223,0.082,-0.067,14.464,14.464,69.137,69.137,1106.186,1106.186,0.000
8,H20,8,144,16,0.341,0.162,0.118,0.051,0.032,0.105,0.031,0.255,0.082,-0.068,16.303,16.303,61.338,61.338,981.406,981.406,0.000
9,H20,8,36,32,0.630,0.212,0.164,0.092,0.050,0.397,0.051,0.611,0.144,-0.070,37.949,37.949,26.351,26.351,843.238,843.238,0.000
